In [ ]:
# Goal
#Create a multi-agent system that translates natural language into SQL commands,
#shows the generated SQL to the user and executes it on a
#SQLite database. The system will maintain session-based memory (Short-Term Memory)
#to keep track of confirmed SQL queries during the current session.


In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
# -----------------------------
# Import necessary libraries for AI Agents and session management
# -----------------------------
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools import FunctionTool
from google.adk.runners import Runner
 

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# -----------------------------
# Configure HTTP retry options for API calls
# -----------------------------
# This ensures the agent retries API requests on temporary failures
# such as rate limits (429) or server errors (500, 503, 504)
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [4]:
import sqlite3


conn = sqlite3.connect("store.db")
cursor = conn.cursor()


cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT,
    price REAL NOT NULL,
    stock INTEGER NOT NULL
);
""")


cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    city TEXT
);
""")

 
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    FOREIGN KEY(customer_id) REFERENCES customers(id),
    FOREIGN KEY(product_id) REFERENCES products(id)
);
""")

conn.commit()
conn.close()
print("Database and tables created successfully!")


Database and tables created successfully!


In [5]:
# connect to DB
conn = sqlite3.connect("store.db")
cursor = conn.cursor()

# -----------------------------
# Insert test data into products
# -----------------------------
products_data = [
    ("Laptop", "Electronics", 1200.0, 15),
    ("Smartphone", "Electronics", 800.0, 25),
    ("Headphones", "Electronics", 150.0, 50),
    ("T-shirt", "Clothing", 25.0, 100),
    ("Jeans", "Clothing", 60.0, 80),
    ("Coffee Maker", "Home Appliances", 100.0, 20),
    ("Blender", "Home Appliances", 70.0, 30),
    ("Book", "Books", 20.0, 200),
    ("Notebook", "Stationery", 5.0, 300),
    ("Pen", "Stationery", 2.0, 500)
]

cursor.executemany("INSERT INTO products (name, category, price, stock) VALUES (?, ?, ?, ?);", products_data)

# -----------------------------
# Insert test data into customers
# -----------------------------
customers_data = [
    ("Alice Johnson", "alice@example.com", "Toronto"),
    ("Bob Smith", "bob@example.com", "Vancouver"),
    ("Charlie Lee", "charlie@example.com", "Calgary"),
    ("David Brown", "david@example.com", "Montreal"),
    ("Eva Green", "eva@example.com", "Ottawa"),
    ("Frank White", "frank@example.com", "Edmonton"),
    ("Grace Black", "grace@example.com", "Halifax"),
    ("Henry King", "henry@example.com", "Winnipeg"),
    ("Isabel Scott", "isabel@example.com", "Quebec City"),
    ("Jack Turner", "jack@example.com", "Saskatoon")
]

cursor.executemany("INSERT INTO customers (name, email, city) VALUES (?, ?, ?);", customers_data)

# -----------------------------
# Insert test data into orders
# -----------------------------
orders_data = [
    (1, 1, 2, "2025-11-01"),
    (2, 2, 1, "2025-11-02"),
    (3, 3, 1, "2025-11-03"),
    (4, 4, 3, "2025-11-04"),
    (5, 5, 4, "2025-11-05"),
    (6, 6, 5, "2025-11-06"),
    (7, 7, 6, "2025-11-07"),
    (8, 8, 7, "2025-11-08"),
    (9, 9, 8, "2025-11-09"),
    (10, 10, 9, "2025-11-10")
]

cursor.executemany("INSERT INTO orders (customer_id, product_id, quantity, order_date) VALUES (?, ?, ?, ?);", orders_data)

#  save and disconnect
conn.commit()
conn.close()

print("Test data inserted successfully!")


Test data inserted successfully!


In [6]:

import pandas as pd

# -----------------------------
# Connect to the SQLite database
# -----------------------------
# The database file 'store.db' contains our tables: products, customers, and orders
conn = sqlite3.connect("store.db")

# -----------------------------
# Display all records in the 'products' table
# -----------------------------
products_df = pd.read_sql_query("SELECT * FROM products;", conn)
print("Products Table:")
print(products_df)
print("\n")

# -----------------------------
# Display all records in the 'customers' table
# -----------------------------
customers_df = pd.read_sql_query("SELECT * FROM customers;", conn)
print("Customers Table:")
print(customers_df)
print("\n")

# -----------------------------
# Display all records in the 'orders' table
# -----------------------------
orders_df = pd.read_sql_query("SELECT * FROM orders;", conn)
print("Orders Table:")
print(orders_df)
print("\n")

# -----------------------------
# Close the database connection
# -----------------------------
conn.close()


Products Table:
   id          name         category   price  stock
0   1        Laptop      Electronics  1200.0     15
1   2    Smartphone      Electronics   800.0     25
2   3    Headphones      Electronics   150.0     50
3   4       T-shirt         Clothing    25.0    100
4   5         Jeans         Clothing    60.0     80
5   6  Coffee Maker  Home Appliances   100.0     20
6   7       Blender  Home Appliances    70.0     30
7   8          Book            Books    20.0    200
8   9      Notebook       Stationery     5.0    300
9  10           Pen       Stationery     2.0    500


Customers Table:
   id           name                email         city
0   1  Alice Johnson    alice@example.com      Toronto
1   2      Bob Smith      bob@example.com    Vancouver
2   3    Charlie Lee  charlie@example.com      Calgary
3   4    David Brown    david@example.com     Montreal
4   5      Eva Green      eva@example.com       Ottawa
5   6    Frank White    frank@example.com     Edmonton
6   7   

In [7]:

# -----------------------------
# SQLite functions as tools
# -----------------------------
def sqlite_query_fn(query: str) -> dict:
    """Execute SQL query on store.db"""
    conn = sqlite3.connect("store.db")
    cur = conn.cursor()
    try:
        cur.execute(query)
        rows = cur.fetchall()
        cols = [d[0] for d in cur.description] if cur.description else []
        return {"columns": cols, "rows": rows, "sql": query}
    except Exception as e:
        return {"error": str(e), "sql": query}
    finally:
        conn.close()

def sqlite_schema_fn() -> dict:
    """Get SQLite database schema"""
    conn = sqlite3.connect("store.db")
    cur = conn.cursor()
    try:
        cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = [t[0] for t in cur.fetchall()]
        schema = {}
        for t in tables:
            cur.execute(f"PRAGMA table_info({t})")
            schema[t] = cur.fetchall()
        return {"schema": schema}
    except Exception as e:
        return {"error": str(e)}
    finally:
        conn.close()

# Convert Python functions to ADK tools
sqlite_query_tool = FunctionTool(func=sqlite_query_fn)
sqlite_schema_tool = FunctionTool(func=sqlite_schema_fn)

# -----------------------------
# Data Developer Agent (sub-agent)
# -----------------------------
Data_developer = LlmAgent(
    name="DataDeveloper",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You generate SQL ONLY.

Before generating SQL, you MUST call the tool `sqlite_schema_fn` to read the database schema.
Rules:
- ALWAYS call the schema tool first.
- After seeing the schema, output a single SQL query.
- Output MUST be ONLY SQL code.
- Do NOT add any text before/after the SQL.
- SQL MUST be executable.
- No explanations.
-Output MUST be ONLY SQL code as text. Do not return function calls or JSON.
""",

    tools=[sqlite_schema_tool]  # Now Data_developer can inspect schema
)

# -----------------------------
# SQL Validator Agent
# -----------------------------
SQL_Validator_Agent = LlmAgent(
    name="SQL_Validator_Agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a SQL Executor Agent.
Steps:
1. Ask Data_developer to generate SQL for the user request.
2. show Sql to user
3. Automatically execute the SQL using sqlite_query_tool.
4. Return a dictionary with two keys:
   - 'sql': the SQL code generated
   - 'result': the query result as rows and columns
5. If execution fails, return the error message and SQL.
""",
    tools=[sqlite_query_tool, AgentTool(agent=Data_developer)]
)

# -----------------------------
# Runner and session
# -----------------------------

session_service = InMemorySessionService()
runner = Runner(agent=SQL_Validator_Agent, app_name="default", session_service=session_service)

print("✅ MCP-style SQLite tools and agents ready!")


✅ MCP-style SQLite tools and agents ready!


In [8]:

# --------------------------------------
# test SQL Validator Agent
# --------------------------------------
import asyncio
async def ask(query_text: str):
    response = await runner.run_debug(query_text)

await ask("List all products with categories .")


 ### Created new session: debug_session_id

User > List all products with categories .


SQL_Validator_Agent > I have generated the SQL query for you.
```sql
SELECT name, category FROM products
```

SQL_Validator_Agent > Here are all the products with their categories:

| name       | category    |
|------------|-------------|
| Laptop     | Electronics |
| Smartphone | Electronics |
| Headphones | Electronics |
| T-shirt    | Clothing    |
| Jeans      | Clothing    |
| Coffee Maker | Home Appliances |
| Blender    | Home Appliances |
| Book       | Books       |
| Notebook   | Stationery  |
| Pen        | Stationery  |



In [10]:
await ask("List all customers and total sales for each.")


 ### Continue session: debug_session_id

User > List all customers and total sales for each.


SQL_Validator_Agent > I have generated the SQL query for you.
```sql
SELECT c.name, SUM(o.quantity * p.price) AS total_sales
FROM customers c
JOIN orders o ON c.id = o.customer_id
JOIN products p ON o.product_id = p.id
GROUP BY c.name;
```

SQL_Validator_Agent > Here are the customers and their total sales:

| name        | total_sales |
|-------------|-------------|
| Alice Johnson | 2400        |
| Bob Smith   | 800         |
| Charlie Lee | 150         |
| David Brown | 75          |
| Eva Green   | 240         |
| Frank White | 500         |
| Grace Black | 420         |
| Henry King  | 140         |
| Isabel Scott| 40          |
| Jack Turner | 18          |



In [11]:
await ask("List all categories and total sales for each.")



 ### Continue session: debug_session_id

User > List all categories and total sales for each.


SQL_Validator_Agent > I have generated the SQL query for you.
```sql
SELECT
  category,
  SUM(price * quantity) AS total_sales
FROM products
JOIN orders
  ON products.id = orders.product_id
GROUP BY
  category;
```

SQL_Validator_Agent > Here are the categories and their total sales:

| category    | total_sales |
|-------------|-------------|
| Books       | 140         |
| Clothing    | 315         |
| Electronics | 3350        |
| Home Appliances | 920         |
| Stationery  | 58          |

